In [1]:
import json
import numpy as np
from sklearn.metrics import cohen_kappa_score

In [2]:
# Função para carregar as anotações a partir de um arquivo JSONL
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        annotations = [json.loads(line) for line in f.readlines()]
    return annotations


In [28]:
# Função para criar uma lista de rótulos tokenizada
def create_labels(text, annotations):
    labels = ["O"] * len(text)  # Lista inicial de rótulos "O" (outside)
    for start, end, label in annotations['label']:
        if start < len(text) and end <= len(text):
            for i in range(start, end):
                labels[i] = label
    return labels


In [4]:
# Função para alinhar anotações pelo ID
def align_by_id(annotator1, annotator2):
    aligned_1 = []
    aligned_2 = []
    for a1 in annotator1:
        for a2 in annotator2:
            if a1['id'] == a2['id']:  # Alinhar pelo ID
                labels_1 = create_labels(a1['text'], a1)
                labels_2 = create_labels(a2['text'], a2)
                aligned_1.append(labels_1)
                aligned_2.append(labels_2)
    return aligned_1, aligned_2

In [5]:
# Função para calcular Kappa de Cohen para todos os pares com verificação de variação
def calculate_kappa_for_all(aligned_1, aligned_2, all_labels):
    kappas = []
    for labels_1, labels_2 in zip(aligned_1, aligned_2):
        # Verifica se existe variação nas anotações
        if len(set(labels_1)) > 1 or len(set(labels_2)) > 1:
            kappa = cohen_kappa_score(labels_1, labels_2, labels=all_labels)
        else:
            kappa = None  # Ignora se não houver variação
        kappas.append(kappa)
    return kappas


In [19]:
# Função para filtrar anotações com base em IDs
def filter_annotations_by_ids(annotations, ids):
    # Filtrar anotações que têm o ID especificado
    filtered_annotations = [annotation for annotation in annotations if annotation['id'] in ids]
    return filtered_annotations

In [6]:
# Definir os rótulos conhecidos (entidades)
all_labels = [
    "O",  # Outside, sem anotação
    "time", "goleiro", "defensor", "meio-campista", "atacante",
    "campeonato", "estádio", "técnico", "árbitro principal", "assistente", "placar"
]

In [10]:
# Carregar os três arquivos de anotadores (JSONL)
annotations_annotator_1 = load_annotations('./data/corpus_annotated/henrique.jsonl')
annotations_annotator_2 = load_annotations('./data/corpus_annotated/florensa.jsonl')
annotations_annotator_3 = load_annotations('./data/corpus_annotated/pedro.jsonl')

In [11]:
# Alinhar anotadores 1 e 2 pelo ID
aligned_1_2_1, aligned_1_2_2 = align_by_id(annotations_annotator_1, annotations_annotator_2)

# Alinhar anotadores 2 e 3 pelo ID
aligned_2_3_2, aligned_2_3_3 = align_by_id(annotations_annotator_2, annotations_annotator_3)

# Alinhar anotadores 1 e 3 pelo ID
aligned_1_3_1, aligned_1_3_3 = align_by_id(annotations_annotator_1, annotations_annotator_3)

In [12]:
# Calcular o Kappa de Cohen para cada par de anotadores com verificação de variação
kappa_1_2 = calculate_kappa_for_all(aligned_1_2_1, aligned_1_2_2, all_labels)
kappa_2_3 = calculate_kappa_for_all(aligned_2_3_2, aligned_2_3_3, all_labels)
kappa_1_3 = calculate_kappa_for_all(aligned_1_3_1, aligned_1_3_3, all_labels)

In [13]:
# Calcular a média do Kappa Cohen para cada par de anotadores
valid_kappas_1_2 = [k for k in kappa_1_2 if k is not None]
mean_kappa_1_2 = np.mean(valid_kappas_1_2)

valid_kappas_2_3 = [k for k in kappa_2_3 if k is not None]
mean_kappa_2_3 = np.mean(valid_kappas_2_3)

valid_kappas_1_3 = [k for k in kappa_1_3 if k is not None]
mean_kappa_1_3 = np.mean(valid_kappas_1_3)

In [14]:
print("Média Kappa de Cohen para 99 textos (Henrique vs Florensa):", mean_kappa_1_2)
print("Média Kappa de Cohen para 99 textos  (Florensa vs Pedro):", mean_kappa_2_3)
print("Média Kappa de Cohen para 99 textos  (Henrique vs Pedro):", mean_kappa_1_3)

Média Kappa de Cohen para 99 textos (Henrique vs Florensa): 0.8263866110497625
Média Kappa de Cohen para 99 textos  (Florensa vs Pedro): 0.5445384430444409
Média Kappa de Cohen para 99 textos  (Henrique vs Pedro): 0.5583901242607661


In [16]:
# Carregar os dois arquivos de llm (JSONL)
annotations_annotator_llm_zs = load_annotations('./data/corpus_annotated/llm_zero_shot.jsonl')
annotations_annotator_llm_fs = load_annotations('./data/corpus_annotated/llm_few_shot.jsonl')

In [18]:
ids_to_filter = [104, 129, 163, 165, 172, 153]

In [24]:
filtered_annotations_annotator_1 = filter_annotations_by_ids(annotations_annotator_1, ids_to_filter)

In [29]:
# Alinhar anotadores 1 e llm zero shot pelo ID
aligned_1_zs_1, aligned_1_zs_2 = align_by_id(filtered_annotations_annotator_1, annotations_annotator_llm_zs)

# Alinhar anotadores 1 e llm few shot pelo ID
aligned_1_fs_1, aligned_1_fs_2 = align_by_id(filtered_annotations_annotator_1, annotations_annotator_llm_fs)

In [30]:
# Calcular o Kappa de Cohen para cada par de anotadores com verificação de variação
kappa_1_zs = calculate_kappa_for_all(aligned_1_zs_1, aligned_1_zs_2, all_labels)
kappa_1_fs = calculate_kappa_for_all(aligned_1_fs_1, aligned_1_fs_2, all_labels)

ValueError: Found input variables with inconsistent numbers of samples: [1419, 1418]

In [ ]:
# Calcular a média do Kappa Cohen para cada par de anotadores
valid_kappas_1_zs = [k for k in kappa_1_zs if k is not None]
mean_kappa_1_zs = np.mean(valid_kappas_1_zs)

valid_kappas_1_fs = [k for k in kappa_1_fs if k is not None]
mean_kappa_1_fs = np.mean(valid_kappas_1_fs)

In [ ]:
print("Média Kappa de Cohen para 99 textos (Henrique vs LLM zero shot):", mean_kappa_1_zs)
print("Média Kappa de Cohen para 99 textos  (Florensa vs LLM few shot):", mean_kappa_1_fs)